In [102]:
import pandas as pd 
import numpy as np

train_data = pd.read_csv('titanic_train.csv')
test_data = pd.read_csv('titanic_test.csv')

In [103]:
#전처리 (성별, Embarked)

train_data.loc[train_data['Sex'] == 'male' , 'Sex'] =0
train_data.loc[train_data['Sex'] == 'female' , 'Sex'] =1
temp_train = pd.get_dummies(train_data['Embarked'], prefix='Embarked')  
train_data = pd.concat([train_data, temp_train] ,axis=1)
train_data = train_data.drop(['Embarked'], axis= 1)


test_data.loc[test_data['Sex'] == 'male' , 'Sex'] =0
test_data.loc[test_data['Sex'] == 'female' , 'Sex'] =1
temp_test = pd.get_dummies(test_data['Embarked'], prefix='Embarked')  
test_data = pd.concat([test_data, temp_test] ,axis=1)
test_data = test_data.drop(['Embarked'], axis= 1)

In [104]:
#전처리 (나이)
train_data.loc[(train_data['Sex']==0) & (train_data['Age'].isna()),'Age'] = train_data.loc[train_data['Sex']==0,'Age'].median()
train_data.loc[(train_data['Sex']==1) & (train_data['Age'].isna()),'Age'] = train_data.loc[train_data['Sex']==1,'Age'].median()

# 나이,Fare 중간값
test_data.loc[(test_data['Sex']==0) & (test_data['Age'].isna()),'Age'] = test_data.loc[test_data['Sex']==0,'Age'].median()
test_data.loc[(test_data['Sex']==1) & (test_data['Age'].isna()),'Age'] = test_data.loc[test_data['Sex']==1,'Age'].median()

test_data.loc[test_data['Fare'].isna(),'Fare'] = test_data['Fare'].median()


In [105]:
from sklearn.model_selection import train_test_split

x_fatures = ['Pclass','Age', 'Sex', 'SibSp', 'Parch', 'Fare','Embarked_C', 'Embarked_Q', 'Embarked_S']

x_train, x_test, y_train, y_test = train_test_split(train_data[x_fatures], train_data['Survived'], stratify = train_data['Survived'], random_state=42)


In [106]:
from sklearn.decomposition import PCA
pca = PCA(n_components=9,whiten= False, random_state=0).fit(x_train)
x_train_pca = pca.transform(x_train)
x_test_pca = pca.transform(x_test)

test = test_data[['Pclass','Age', 'Sex', 'SibSp', 'Parch', 'Fare','Embarked_C', 'Embarked_Q', 'Embarked_S']]
test_pca = pca.transform(test)
# print(x_train_pca.shape)

In [100]:
## 수행
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(n_estimators=500, max_depth=5,
                              random_state=0, n_jobs=4)
# clf2 = RandomForestClassifier(n_estimators=500, max_depth=5,
#     random_state=0, n_jobs=4)
clf.fit(x_train, y_train)
print('scale전 : ',clf.score(x_train,y_train))
print('scale전_TEST : ',clf.score(x_test,y_test))
clf.fit(x_train_pca,y_train)
# clf2.fit(x_train,y_train)
print('scale후 : ', clf.score(x_train_pca,y_train))
print('scale후_TEST : ',clf.score(x_test_pca,y_test))


# grid_search = GridSearchCV(RandomForestClassifier(), param_grid, cv=5,return_train_score =True )
# grid_search.fit(x_train,y_train)
# # print("테스트 세트 점수: {:.2f}".format(grid_search.score(x_test, y_test)))
# print("최적 매개변수:", grid_search.best_params_)
# print("최고 교차 검증 점수: {:.2f}".format(grid_search.best_score_))
# print("최고 성능 모델:\n", grid_search.best_estimator_)


# pred = grid_search.predict(x_test)
pred = clf.predict(test_pca)
print(clf.score(test_pca,pred))

# sub = pd.DataFrame(columns=[test_data['PassengerId'],pred])

# sub.to_csv('Submit_titanic.csv', sep=',', encoding ='utf-8')

sub = pd.DataFrame()
sub['PassengerId']=test_data['PassengerId']
sub['Survived'] = pred

sub.index = sub['PassengerId']
sub.drop(columns='PassengerId', inplace=True)
sub.to_csv('Submit_titanic.csv', sep=',', encoding ='utf-8')

scale전 :  0.8667664670658682
scale전_TEST :  0.8026905829596412
scale후 :  0.8862275449101796
scale후_TEST :  0.8026905829596412
1.0


In [111]:
## m수행
import xgboost as xgb
xgbClf = xgb.XGBClassifier(colsample_bytree = 0.3, learning_rate = 0.1,
                max_depth = 5, alpha = 10, n_estimators = 10)


xgbClf.fit(x_train_pca, y_train)
print('scale전 : ',xgbClf.score(x_train,y_train))
print('scale전_TEST : ',xgbClf.score(x_test,y_test))

xgbClf.fit(x_train, y_train)
print('scale후 : ', xgbClf.score(x_train_pca,y_train))
print('scale후_TEST : ',xgbClf.score(x_test_pca,y_test))


# grid_search = GridSearchCV(RandomForestClassifier(), param_grid, cv=5,return_train_score =True )
# grid_search.fit(x_train,y_train)
# # print("테스트 세트 점수: {:.2f}".format(grid_search.score(x_test, y_test)))
# print("최적 매개변수:", grid_search.best_params_)
# print("최고 교차 검증 점수: {:.2f}".format(grid_search.best_score_))
# print("최고 성능 모델:\n", grid_search.best_estimator_)


# pred = grid_search.predict(x_test)
pred = xgbClf.predict(test_pca)
print(xgbClf.score(test_pca,pred))

# sub = pd.DataFrame(columns=[test_data['PassengerId'],pred])

# sub.to_csv('Submit_titanic.csv', sep=',', encoding ='utf-8')

sub = pd.DataFrame()
sub['PassengerId']=test_data['PassengerId']
sub['Survived'] = pred

sub.index = sub['PassengerId']
sub.drop(columns='PassengerId', inplace=True)
sub.to_csv('Submit_titanic.csv', sep=',', encoding ='utf-8')

ValueError: feature_names mismatch: ['f0', 'f1', 'f2', 'f3', 'f4', 'f5', 'f6', 'f7', 'f8'] ['Pclass', 'Age', 'Sex', 'SibSp', 'Parch', 'Fare', 'Embarked_C', 'Embarked_Q', 'Embarked_S']
expected f0, f2, f7, f5, f8, f4, f3, f1, f6 in input data
training data did not have the following fields: Embarked_S, SibSp, Fare, Parch, Embarked_C, Embarked_Q, Pclass, Sex, Age